# PG Databases

In [1]:
import psycopg2
import pandas as pd
import configparser
import matplotlib.pyplot as plt

# Read from the Config file
config = configparser.ConfigParser()
config.read_file(open(r'../db_ops.cfg'))

con_str = config.get('con_str', 'PG_AIRBASES') 



In [4]:
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from psycopg2 import OperationalError

def create_connection(con_str):
    """
    Creates a connection to the database using SQLAlchemy.
    
    Args:
    con_str (str): The connection string for the database.

    Returns:
    connection: The connection object if successful, None otherwise.
    """
    try:
        # Create a connection using the connection string
        engine = create_engine(con_str)
        connection = engine.connect()
        print("Opened Connection")
        return connection
    except OperationalError as e:
        print("OperationalError: Could not connect to the database. Please check the connection details and ensure the database server is running.")
        print(f"Details: {e}")
    except SQLAlchemyError as e:
        print("SQLAlchemyError: An error occurred while connecting to the database.")
        print(f"Details: {e}")
    except Exception as e:
        print("An unexpected error occurred while connecting to the database.")
        print(f"Details: {e}")
    
    return None

In [3]:

sql_command = """
SELECT relid,
		schemaname, 
    relname as table_name, 
    (schemaname || '.' || relname) as full_table_name,
    n_live_tup as rows,
    n_dead_tup as dead_rows,
    n_mod_since_analyze,
    case 
    	when n_live_tup = 0 THEN 0
      else  n_mod_since_analyze / n_live_tup 
      end as pct_mod_since_analyze,
    last_analyze as last_analyze_date,
    last_autoanalyze as last_autoanalyze_date,
    pg_total_relation_size(relid) / 1024 as total_table_size_kb, 
    pg_table_size(relid) / 1024 as table_size_kb,
    pg_indexes_size(relid) / 1024 as indexes_size_kb,
    pg_size_pretty(pg_total_relation_size(relid)) as total_table_size_pretty, 
    pg_size_pretty(pg_table_size(relid)) as table_size_pretty,
    pg_size_pretty(pg_indexes_size(relid)) as index_size_pretty
    
FROM pg_stat_user_tables
ORDER BY schemaname, 
    relname
"""

try:
    connection = create_connection(con_str);
    # Execute the SQL command
    df = pd.read_sql_query(sql_command, connection)
    # Set the display options to show all columns without truncation
    pd.set_option('display.max_columns', None)
    pd.set_option('display.expand_frame_repr', False)   
    print(df)
except (SQLAlchemyError, ValueError) as e:
    # Handle any errors or raised exceptions
    raise e

Opened Connection
       relid    schemaname                         table_name                          full_table_name      rows  dead_rows  n_mod_since_analyze  pct_mod_since_analyze                last_analyze_date            last_autoanalyze_date  total_table_size_kb  table_size_kb  indexes_size_kb total_table_size_pretty table_size_pretty index_size_pretty
0   27071560      airbases                            flights                         airbases.flights   1085084       6860                 4216                      0                              NaT 2024-07-23 03:20:13.348623+00:00               106040          82216            23824                  104 MB             80 MB             23 MB
1   27071569      airbases                          log_table                       airbases.log_table     20883          0                  220                      0                              NaT 2024-07-22 05:20:01.316306+00:00                  936            936                0  

In [ ]:
## Close the connection


# finally: # Close the connection
#    if connection:
#        connection.close()
#        print("Connection closed")